In [1]:
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
# Load Data 
X,y = shap.datasets.adult()
X_display,y_display = shap.datasets.adult(display=True) # human readable feature values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)


In [3]:
#Model

In [4]:
scm = CausalModel("census")
dataset = scm.generate_dataset(10000)

display(dataset.df)
#scm

Census Model Initiate
Structural Equation Finished
Noise Distribution Finished
({'x1': <function census_model.<locals>.<lambda> at 0x7f3000c8c200>, 'x2': <function census_model.<locals>.<lambda> at 0x7f3000c8c290>, 'x3': <function census_model.<locals>.<lambda> at 0x7f3000c8c320>, 'x4': <function census_model.<locals>.<lambda> at 0x7f3000c8c3b0>, 'x5': <function census_model.<locals>.<lambda> at 0x7f3000c8c440>, 'x6': <function census_model.<locals>.<lambda> at 0x7f3000c8c4d0>, 'x7': <function census_model.<locals>.<lambda> at 0x7f3000c8c560>, 'x8': <function census_model.<locals>.<lambda> at 0x7f3000c8c5f0>, 'x9': <function census_model.<locals>.<lambda> at 0x7f3000c8c680>}, {'x1': <function census_model.<locals>.<lambda> at 0x7f3000c8c200>, 'x2': <function census_model.<locals>.<lambda> at 0x7f3000c8c290>, 'x3': <function census_model.<locals>.<lambda> at 0x7f3000c8c320>, 'x4': <function census_model.<locals>.<lambda> at 0x7f3000c8c3b0>, 'x5': <function census_model.<locals>.<lambda>

,label,x1,x2,x3,x7,x8,x4,x5,x6,x9
0,1.0,0.159288,-1.563031,0.385003,-0.172256,-0.845529,-0.352016,-0.495807,0.014625,0.343291
1,1.0,-3.311208,0.519718,1.273104,0.473927,0.787613,0.959121,-0.091338,2.257265,0.123564
2,0.0,-2.557423,-0.292332,-0.175909,-1.688535,0.187304,1.400227,0.149817,-0.961060,-0.096228
3,1.0,0.936960,0.018228,0.337136,-0.843082,0.681472,0.241646,0.246484,-1.149778,-2.030897
4,1.0,2.216905,-0.975050,0.106965,-0.172383,2.500239,0.833125,-1.231935,-1.507751,-0.475264
...,...,...,...,...,...,...,...,...,...,...
9995,0.0,-1.303996,0.252804,-0.217440,0.049078,-1.584153,0.329447,-0.433709,1.260422,-0.941562
9996,1.0,-1.788502,0.884365,0.264603,-0.510517,-1.692986,1.788382,0.518203,0.460668,-0.180115
9997,0.0,0.272844,0.620954,2.062550,-2.102655,0.404494,-0.881793,0.958360,-0.706788,1.905003
9998,1.0,1.309561,0.181473,0.506388,-0.779774,-0.029753,-1.268766,-0.140942,0.505844,-0.442167


In [5]:
from carla.models.catalog import MLModelCatalog


training_params = {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 3]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)

balance on test set 0.4528, balance on test set 0.4408
Epoch 0/9
----------
train Loss: 0.3723 Acc: 0.8335

test Loss: 0.3489 Acc: 0.8460

Epoch 1/9
----------
train Loss: 0.3618 Acc: 0.8375

test Loss: 0.3408 Acc: 0.8488

Epoch 2/9
----------
train Loss: 0.3586 Acc: 0.8403

test Loss: 0.3579 Acc: 0.8420

Epoch 3/9
----------
train Loss: 0.3577 Acc: 0.8400

test Loss: 0.3404 Acc: 0.8488

Epoch 4/9
----------
train Loss: 0.3554 Acc: 0.8405

test Loss: 0.3500 Acc: 0.8472

Epoch 5/9
----------
train Loss: 0.3556 Acc: 0.8397

test Loss: 0.3518 Acc: 0.8380

Epoch 6/9
----------
train Loss: 0.3550 Acc: 0.8396

test Loss: 0.3450 Acc: 0.8492

Epoch 7/9
----------
train Loss: 0.3534 Acc: 0.8415

test Loss: 0.3531 Acc: 0.8420

Epoch 8/9
----------
train Loss: 0.3525 Acc: 0.8420

test Loss: 0.3449 Acc: 0.8444

Epoch 9/9
----------
train Loss: 0.3528 Acc: 0.8417

test Loss: 0.3514 Acc: 0.8432



In [6]:
from carla.models.negative_instances import predict_negative_instances
from carla.recourse_methods.catalog.causal_recourse import (
    CausalRecourse,
    constraints,
    samplers,
)


In [7]:
# get factuals
factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:5]

In [8]:
hyperparams = {
    "optimization_approach": "brute_force",
    "num_samples": 10,
    "scm": scm,
    "constraint_handle": constraints.point_constraint,
    "sampler_handle": samplers.sample_true_m0,
}
cfs = CausalRecourse(ml_model, hyperparams).get_counterfactuals(test_factual)

display(cfs)

: 

: 